In [1]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [2]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("disable-gpu")

## Get xy coordinate directly from embeded navermap

In [208]:
chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)
url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1001&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1'
# url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?pageUnit=2500&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1'
driver.get(url) 

time.sleep(10)

source = driver.page_source 
data = bs(source, 'html.parser')

In [305]:
cont = data.findAll("div", {"class":"map_list"})[0]

In [483]:
ul = cont.ul.select("li[onclick]")

In [492]:
len(ul)

1001

In [523]:
cols = ['xy','name','addr', 'call_num']
df = pd.DataFrame(columns=cols)

for i in range(len(ul)):
    camp = ul[i]
    xy = camp['onclick']
    name = camp.h2.text.split(']')[1].strip()
    addr = camp.find("li", {"class":"addr"}).get_text()
    call_num = camp.find("li", {"class":"call_num"}).get_text() if camp.find("li", {"class":"call_num"}) else ""
    val_list = [xy, name, addr, call_num]
    result = dict(zip(cols, val_list))
    df = df.append(result, ignore_index=True)

In [524]:
df.head(10)

,xy,name,addr,call_num
0,"markerClick(marker_0, infowindow_0, '37.256823...",트리캠핑장,인천광역시 옹진군 선재로306번길 27-55,
1,"markerClick(marker_1, infowindow_1, '35.642447...",마노아워터캠핑장,경남 합천군 봉산면 압곡3길 9,010-8667-6051
2,"markerClick(marker_2, infowindow_2, '36.045565...",서천 금빛노을 서울캠핑장,충청남도 서천군 장선리 128-3,
3,"markerClick(marker_3, infowindow_3, '37.727152...",수동시온캠프,경기 남양주시 수동면 지둔로445번길 82,010-5230-6729
4,"markerClick(marker_4, infowindow_4, '35.530722...",아트앤오토캠핑장,울산광역시 울주군 상북면 이천리,010-3710-2060
5,"markerClick(marker_5, infowindow_5, '37.355251...",영월지구별캠프,강원 영월군 무릉도원면 무릉법흥로 1104,010-7923-4498
6,"markerClick(marker_6, infowindow_6, '37.514232...",힐링스토리,강원 횡성군 서원면 금대1길 24-68,010-5227-3579
7,"markerClick(marker_7, infowindow_7, '35.493419...",밀양댐오토캠핑장,경상남도 밀양시 단장면 고례리,010-5199-1426
8,"markerClick(marker_8, infowindow_8, '34.917515...",별볼일 있는 야영장,전라남도 보성군 문덕면 양동리 942,
9,"markerClick(marker_9, infowindow_9, '37.125710...",별마로빌리지캠핑장,강원 영월군 김삿갓면 영월동로 1954-40,010-3008-6408


## geocoding example

In [ ]:
cols = list(df.columns) + ['level1','level2','x','y']
df_fn = pd.DataFrame(columns=cols)
apikey = '6829FE20-AE45-312E-9D0A-418C7E77BF47'

for i in range(len(df)):
    dict0 = dict(df.iloc[i])
#     print(dict0)

    address = df['addr'][i]
    URL = 'http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={addr}&refine=true&simple=false&format=json&type=road&key={key}'.format(key=apikey, addr=address)
    r = requests.get(URL)
    data = r.json()
    try: 
        dict1 = data['response']['refined']['structure']
        dict1 = {k: dict1[k] for k in dict1.keys() & {'level1','level2'}}
        dict2 = data['response']['result']['point']
        dict0.update(dict1)
        dict0.update(dict2)
    except KeyError:
        dict1 = {'level1':None, 'level2':None, 'x':None, 'y':None}
        dict0.update(dict1)
    
    df_tmp = pd.DataFrame(json_normalize(dict0), columns=dict0.keys())
    df_fn = df_fn.append(df_tmp)
    print(df_fn.tail(5))